# Connected components

Решение, основанное на анализе компонент связности

In [ ]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from pathlib import Path

In [ ]:
data_path = Path("./data")
images_path = data_path / "competition/competition/"
masks_path  = data_path / "masks/"

In [ ]:
from utils import plot_with_mask
from utils_uint8 import *

In [ ]:
def get_mask(image: np.array, min_component_area: int=150) -> np.array:
    """
    Получение маски.
    
    Параметры
    ---------
    image : np.array
        Изображение.
    min_component_area : int
        Минимальная площадь компоненты связности.
    """
    
    brain_mask = get_brain_mask(image)
    image = np.minimum(image, brain_mask)
    
    # Фильтрация шума.
    image = cv2.bilateralFilter(image, 9, 10, 75)
    
    # Выделение первоначальной маски.
    _, mask = cv2.threshold(image, 40, 255, cv2.THRESH_BINARY)
    
    # Морфологические преобразования для фильтрации шума в маске.
    kernel_size = 2
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (kernel_size, kernel_size))
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    
    kernel_size = 2
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (kernel_size, kernel_size))
    mask = cv2.dilate(mask, kernel, iterations=2)
    
    mask = np.minimum(mask, brain_mask)
    
    # Получение компонент связности.
    n_components, components_mask, stats, centroids = cv2.connectedComponentsWithStats(mask, connectivity=8)
    
    # Построение итоговой маски.
    mask.fill(0)
    for index in range(1, n_components):
        # Проход по всем компонентам.
        
        # Площадь.
        component_area = stats[index, cv2.CC_STAT_AREA]
        
        # Отсев компонент по площади.
        if component_area > min_component_area:
            # Маска текущей компоненты.
            component_mask = np.zeros_like(mask)
            component_mask[components_mask == index] = 255
            
            #plot_with_mask(image, component_mask)
            
            # Выделение ближайших к границе укружающих и внутренних пикселей.
            kernel_size = 5
            kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (kernel_size, kernel_size))
            dilated_mask = np.minimum(cv2.dilate(component_mask, kernel), brain_mask)
            eroded_mask  = cv2.erode(component_mask, kernel)
            
            # Вычисление трёх характеристик.
            mean_mask = np.median(image[component_mask == 255])
            mean_delta_dilated = np.median(image[(dilated_mask == 255) & (component_mask == 0)])
            mean_delta_eroded  = np.median(image[(eroded_mask  == 0)   & (component_mask == 255)])
            
            #print(mean_mask, mean_delta_eroded, mean_delta_dilated)
            
            # Вот тут надо бы поподбирать. МБ даже имеет смысл кластеризацию какую-то делать.
            # Текущая идея: смотрим на контраст соседних к границе пикселей.
            if mean_mask > mean_delta_eroded + 4.0:
                mask[components_mask == index] = 255
    
    return mask

In [ ]:
ious = []

for index in range(5):
    name = f"{index:06d}.jpg"
    image_path = images_path / name
    mask_path  = masks_path  / name
    
    image = cv2.imread(str(image_path), cv2.IMREAD_GRAYSCALE)
    true_mask = cv2.imread(str(mask_path), cv2.IMREAD_GRAYSCALE)
    
    mask = get_mask(image)
    plot_with_mask(image, mask)
    
    intersection_over_union = np.count_nonzero(np.minimum(mask, true_mask)) / np.count_nonzero(np.maximum(mask, true_mask))
    ious.append(intersection_over_union)

In [ ]:
sum(ious) / 5

In [ ]:
for index in range(26600, 26725):
    print(index)
    name = f"{index:06d}.jpg"
    image_path = images_path / name
    
    image = cv2.imread(str(image_path), cv2.IMREAD_GRAYSCALE)
    
    mask = get_mask(image)
    plot_with_mask(image, mask)
    
    plt.close()

In [ ]:
def df_from_mask(name, mask):
    numbers = pd.Series(np.arange(mask.size), name = 'ID')
    ind = numbers.apply(lambda n: name + f"_{n // mask.shape[1]}_{n % mask.shape[1]}")
    return pd.DataFrame({'value': mask.flatten()}, index = ind, dtype=np.int32)

In [ ]:
from tqdm import tqdm

dataframes = []

for index in tqdm(range(26600, 26725)):
    name = f"{index:06d}"
    image_path = images_path / (name + ".jpg")
    
    image = cv2.imread(str(image_path), cv2.IMREAD_GRAYSCALE)
    
    mask = get_mask(image).astype(bool)
    
    #answer = pd.concat([answer, df_from_mask(name, mask)])
    dataframes.append(df_from_mask(name, mask))
    
    #fig = plt.figure(figsize=(10, 10))
    #plt.imshow(mask)
    #plt.show()

In [ ]:
answer = pd.concat(dataframes)
answer.to_csv('connected_components.csv')